## Introduction

In this note-book we will use unseen data to validate our model. This data set was never used during the process of training and testing 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
col=[
'duration',
'protocol_type',
'service',
'flag' ,
'src_bytes',
'dst_bytes',
'land',
'wrong_fragment',
'urgent',
'hot',
'num_failed_logins',
'logged_in',
'num_compromised',
'root_shell',
'su_attempted',
'num_root',
'num_file_creations',
'num_shells',
'num_access_files',
'num_outbound_cmds',
'is_host_login',
'is_guest_login',
'count',
'srv_count',
'serror_rate',
'srv_serror_rate',
'rerror_rate',
'srv_rerror_rate',
'same_srv_rate',
'diff_srv_rate',
'srv_diff_host_rate',
'dst_host_count',
'dst_host_srv_count',
'dst_host_same_srv_rate',
'dst_host_diff_srv_rate',
'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate',
'dst_host_serror_rate',
'dst_host_srv_serror_rate',
'dst_host_rerror_rate',
'dst_host_srv_rerror_rate',
'labels'
]

In [3]:
val_file=pd.read_csv('./Data/validation', header=None, names=col)

In [4]:
val_df=pd.DataFrame(val_file)

In [5]:
val_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels
0,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
3,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,snmpgetattack.
4,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,snmpgetattack.


In [6]:
# FUNCTION 1 DATAPROFILE
shp=val_df.shape
shape=pd.DataFrame(index=None)
shape['num_rows']=np.array([shp[0]])
shape['num_cols']=np.array([shp[1]])
shape

,num_rows,num_cols
0,311029,42


## Checking if there is any missing values

In [8]:
val_df.isnull().sum().sum()

0

In [82]:
# FUNCTION 2 DATATYPE
data_types=[val_df[i].dtypes for i in val_df.columns ]
type_dict={}
for i in ['object', 'int64', 'float64']:
    type_dict[i]=[data_types.count(i)]
    

pd.DataFrame.from_dict(type_dict) 

,float64,int64,object
0,15,23,4


# DATA ENGINEERING

In [9]:

def dtype_transform(df, dt):
    int_types = ["int8", "int16", "int32", "int64"]
    float_types=["float16", "float32", "float64", "float128"]
    dmax=0
    dmin=0
    lmd_floatmax=lambda x: np.finfo(x).max
    lmd_floatmin=lambda x: np.finfo(x).min
    lmd_intmax=lambda x: np.iinfo(x).max
    lmd_intmin=lambda x: np.iinfo(x).min
    max_funct=None
    min_funct=None
    dt_types=[]
    if dt==float:
        dt_types=float_types
        max_funct=lmd_floatmax
        min_funct=lmd_floatmin
    elif dt==int:
        dt_types=int_types
        max_funct=lmd_intmax
        min_funct=lmd_intmin
    else:
        
        exit()
        
    selected_types=None    
    for i in df.columns:
        if df[i].dtypes==dt:
            
            dmax=df[i].max()
            dmin=df[i].min()
            for j in dt_types:
                if max_funct(j)>dmax and min_funct(j)<dmin:
                    
                    selected_types=j
                    
                    
                    break
                else:
                    
                    continue
            
            df[i]=df[i].astype(selected_types)

In [10]:
dtype_transform(val_df, float)
dtype_transform(val_df, int)

# DATA CLEANING

In [11]:
val_df['labels']=val_df['labels'].map(lambda x: x.replace('.',''))

In [12]:
# Attack type `1
a1=[
'Apache2',
'arppoison', 
'Back',
'Crashiis',
'dosnuke', 
'Land',
'Mailbomb',
'worm',
'Mailbomb',
'neptune', 
'PingofDeath', 
'ProcessTable',
'selfping',
'Smurf',
'sshprocesstable',
'Syslogd',
'tcpreset',
'Teardrop',
'Udpstorm',
'pod'
]
# Attack type `2
a2=[
'anypw',
'casesen',
'Eject',
'Ffbconfig',
'Fdformat',
'Loadmodule', 
'ntfsdos',
'Perl',
'sqlattack',
'Ps',
'sechole',
'Xterm',
'yaga',
'buffer_overflow',
'rootkit'
]
# Attack type `3
a3=[
'Dictionary',
'Ftp_write',
'Guest',
'Httptunnel',
'Imap',
'Named',
'ncftp' ,
'netbus'
'netcat',
'Phf',
'ppmacro', 
'Sendmail',
'sshtrojan', 
'Xlock',
'Xsnoop',
'warezmaster',
'warezclient',
'guess_passwd',
'snmpgetattack',
'snmpguess',
'spy'
]

# Attack type `4
a4=[
'insidesniffer',
'Ipsweep',
'ls_domain', 
'Mscan',
'NTinfoscan',
'Nmap',
'queso' ,
'resetscan', 
'Saint',
'Satan',
'portsweep',
'multihop'
]
dos=[i.lower() for i in a1]
u2ra=[i.lower() for i in a2]
r2ua=[i.lower() for i in a3]
probes=[i.lower() for i in a4]

In [13]:
def atk_type(lbl):
   
    if lbl in dos:
        #print(lbl+" DOS")
        return 'DOS'
    elif lbl in u2ra:
        return 'U2RA'
    elif lbl in r2ua:
        return 'R2LA'
    elif lbl in probes:
        return 'PROBES'
    elif lbl=='normal':
        return 'safe'
    else:
        return np.nan
    
val_df['attack_type']=val_df['labels'].map(lambda x: atk_type(x) )


In [14]:
for lbl in val_df['labels']:
    if lbl in dos:
        pass
    elif lbl in u2ra:
        pass
    elif lbl in r2ua:
       # print(f'{lbl} in R2UA')
       pass
    elif lbl in probes:
        #print(f'{lbl} in Probes')
        pass
    elif lbl=='normal':
        pass
    else:
        print(f'{lbl} Missing')

In [15]:
val_df1=val_df[(val_df['attack_type']=='DOS') | (val_df['attack_type']=='safe')]

In [16]:
drop=['hot','num_failed_logins',  'logged_in', 'num_compromised',
     'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
      'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
     'is_guest_login']

In [17]:
val_df1.drop(columns=drop, inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
val_df1.to_csv('./Data/val1.csv', index=False)

## Summary:
We successfully ingested new dataset, optimized it's memory foot print and performed data cleaning. We saved the data for feature selection, We will use same feature engineering we used during training/testing 